In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/resume/other/default/1/test.pdf


In [2]:
!pip install dotenv "unstructured[pdf]" langchain-community transformers langchain_huggingface faiss-gpu-cu12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 MB 37.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.4/441.4 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.7/367.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 81.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/1

In [3]:
import os
from dotenv import load_dotenv
from unstructured.partition.auto import partition
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.memory import ConversationBufferMemory

2025-07-06 19:41:49.588982: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751830909.772312      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751830909.826037      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
hf_token = '<HF_API_KEY>'

In [5]:
pdf_path = "/kaggle/input/resume/other/default/1/test.pdf"

In [6]:
elements = partition(pdf_path)
full_text = "\n\n".join([el.text for el in elements if el.text.strip()])

In [7]:
full_text

'PRANAV AGRAWAL\n\n(cid:131) +xx-xxxxxxxxxx # xxxxx@gmail.com (cid:239) linkedin.com/in/xxx\n\n— leetcode.com/xxx § github.com/xxx\n\nExperience\n\nThe Indian Express Pvt Ltd ª Data Scientist\n\nApr 2024 – Present Noida, Uttar Pradesh\n\nDeveloped a Personalized Notiﬁcation System (PNS) that delivers user-speciﬁc content, leading to an increase of 20,000+ daily user clicks through improved personalization.\n\nDesigning an AI research agent leveraging a RAG model to extract information from 5+ years of historical data across various channels to assist authors in content research.\n\nImplemented a QR Report generation system using PySpark, automating the distribution of over 800+ performance summaries for articles to both authors and their managers.\n\nBuilt a scalable Competition Crawler to scrape data from 100+ competitor websites, storing it in MySQL with key insights visualized via Superset for downstream analysis by editorial teams.\n\nDesigned Trends Pulse by scraping Google Trends

In [8]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.create_documents([full_text])

In [9]:
docs

[Document(metadata={}, page_content='PRANAV AGRAWAL\n\n(cid:131) +xx-xxxxxxxxxx # xxxxx@gmail.com (cid:239) linkedin.com/in/xxx\n\n— leetcode.com/xxx § github.com/xxx\n\nExperience\n\nThe Indian Express Pvt Ltd ª Data Scientist\n\nApr 2024 – Present Noida, Uttar Pradesh\n\nDeveloped a Personalized Notiﬁcation System (PNS) that delivers user-speciﬁc content, leading to an increase of 20,000+ daily user clicks through improved personalization.'),
 Document(metadata={}, page_content='Designing an AI research agent leveraging a RAG model to extract information from 5+ years of historical data across various channels to assist authors in content research.\n\nImplemented a QR Report generation system using PySpark, automating the distribution of over 800+ performance summaries for articles to both authors and their managers.'),
 Document(metadata={}, page_content='Built a scalable Competition Crawler to scrape data from 100+ competitor websites, storing it in MySQL with key insights visualiz

In [10]:
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding)
retriever = vectorstore.as_retriever()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    token=hf_token,
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [18]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.1,
)

Device set to use cuda:0


In [19]:
llm = HuggingFacePipeline(pipeline=pipe)

In [20]:
custom_task = "resume feedback"
instruction = (
    "You are a professional career coach and resume expert. "
    "Your only task is to provide clear, constructive, and detailed feedback on resumes. "
    "Give suggestions for improvement, formatting, clarity, impact, and alignment with job roles. "
    "Do not answer anything outside resume evaluation."
)

In [21]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    verbose=True
)

In [22]:
chat_history = []
while True:
    query = input("🧑 You (Ask for resume feedback): ")
    if query.strip().lower() in {"exit", "quit"}:
        break

    final_query = f"{instruction}\n\nUser Request:\n{query}"

    response = qa_chain.invoke({
        "question": final_query,
        "chat_history": chat_history
    })

    print(f"🤖 Bot (Resume Coach): {response['answer']}\n")
    chat_history.append((query, response["answer"]))


🧑 You (Ask for resume feedback):  provide feedback


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Built a scalable Competition Crawler to scrape data from 100+ competitor websites, storing it in MySQL with key insights visualized via Superset for downstream analysis by editorial teams.

Designed Trends Pulse by scraping Google Trends and processing competitor articles, extracting topical insights via NLP techniques to guide data-driven editorial planning.

Designing an AI research agent leveraging a RAG model to extract information from 5+ years of historical data across various channels to assist authors in content research.

Implemented a QR Report generation system using PySpark, automating the distribution of over 800+ performance summaries for articles to both authors and their managers.

Education

Noida Instit

🧑 You (Ask for resume feedback):  exit
